In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline


datos = np.genfromtxt('framingham.csv', delimiter=',')
data = pd.read_csv(r'framingham.csv')

data.head()



In [ ]:
print(datos)


In [ ]:

print(datos[0,:])

In [ ]:
data.describe()


In [ ]:
sb.pairplot(data)

In [ ]:
X = data.iloc[:, [0, 1, 2]].values
y = data.iloc[:, 3].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)